In [1]:
# 필요한 라이브러리 설치
!pip install xmltodict

/usr/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=210264) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


# KIPRIS 기반 반도체 장비 AI 특허 데이터셋 구축 (Target-Prior Art Pair)

이 노트북은 **"반도체 장비 분야의 AI 활용 특허"**를 검색하고, 해당 특허의 **"심사관 인용 선행기술(Prior Art)"** 정보를 수집하여 **(Target Patent, Ground Truth Prior Art)** 쌍으로 구성된 데이터셋을 구축합니다.

이 데이터셋은 향후 **Agentic AI 기반 선행기술 조사 방법론**의 성능(Recall/Precision)을 검증하는 실험 데이터로 활용됩니다.

### 주요 단계
1. **환경 설정**: KIPRIS API Key 로드
2. **검색 쿼리 정의**: 반도체 장비(H01L 등) + AI 키워드 조합
3. **특허 검색 (Target 수집)**: `getAdvancedSearch` (항목별 검색) 활용
4. **인용 문헌 수집 (Ground Truth 수집)**: `getBibliographyDetailInfoSearch` (서지상세정보) 활용
5. **데이터셋 구축 및 저장**: JSONL 포맷으로 저장 (`data/processed/`)

In [2]:
import os
import time
import json
import requests
import pandas as pd
from pathlib import Path
from typing import Dict, List, Any, Optional
from dotenv import load_dotenv
from tqdm.auto import tqdm

# 1. 환경 설정 및 API 키 로드
def find_repo_root(start: Path | None = None) -> Path:
    cur = (start or Path.cwd()).resolve()
    for p in (cur, *cur.parents):
        if (p / "pyproject.toml").exists() and (p / "data").exists():
            return p
    return cur

REPO_ROOT = find_repo_root()
load_dotenv(REPO_ROOT / "env")
load_dotenv(REPO_ROOT / ".env")

KIPRIS_API_KEY = os.getenv("KIPRIS_API_KEY", "")
if not KIPRIS_API_KEY:
    raise ValueError("KIPRIS_API_KEY가 설정되지 않았습니다. env 파일을 확인하세요.")

# KIPRIS API 기본 설정
BASE_URL = "http://plus.kipris.or.kr/kipo-api/kipi/patUtiModInfoSearchSevice"

# 오퍼레이션 경로
# 1) 항목별 검색 (Target 특허 검색용)
PATH_ADVANCED_SEARCH = "getAdvancedSearch"
# 2) 서지상세정보 (인용 문헌 확인용)
PATH_BIBLIO_DETAIL = "getBibliographyDetailInfoSearch"

print(f"API Key Loaded: {KIPRIS_API_KEY[:4]}... (Length: {len(KIPRIS_API_KEY)})")
print(f"Repo Root: {REPO_ROOT}")

API Key Loaded: aggT... (Length: 44)
Repo Root: /home/arkwith/Dev/paper_data


In [3]:
def kipris_get(path: str, params: Dict[str, Any], max_retries: int = 3) -> Dict[str, Any]:
    """KIPRIS API GET 요청 래퍼 (재시도 및 에러 처리 포함)"""
    url = f"{BASE_URL}/{path}"
    params = params.copy()
    params["ServiceKey"] = KIPRIS_API_KEY
    
    # KIPRIS는 기본적으로 XML을 반환할 수 있으므로, JSON 포맷 요청이 가능한지 확인 필요하지만,
    # 공공데이터포털 연계 API는 보통 XML이 기본입니다. 여기서는 requests가 텍스트로 받아 처리합니다.
    # (일부 오퍼레이션은 _type=json 파라미터를 지원하기도 함)
    
    for attempt in range(max_retries):
        try:
            resp = requests.get(url, params=params, timeout=30)
            resp.raise_for_status()
            
            # 응답이 XML인 경우와 JSON인 경우를 구분해야 함
            # 여기서는 편의상 xmltodict 등을 쓰지 않고, KIPRIS가 제공하는 JSON 옵션이 있다면 활용하거나
            # 간단한 파싱 로직을 사용합니다. (KIPRIS Plus는 보통 XML 응답)
            
            # 만약 JSON 변환이 지원된다면:
            # return resp.json()
            
            # XML을 dict로 변환하기 위해 xmltodict 라이브러리가 필요할 수 있음
            # 여기서는 간단히 xmltodict를 사용한다고 가정 (없으면 설치 필요)
            import xmltodict
            return xmltodict.parse(resp.text)
            
        except Exception as e:
            if attempt == max_retries - 1:
                print(f"Request Failed: {url} | Error: {e}")
                return {}
            time.sleep(1 * (attempt + 1))
    return {}

In [4]:
# 2. 검색 쿼리 정의 (반도체 장비 + AI)
# IPC 코드: H01L (반도체 장치), G06N (컴퓨터 시스템/AI) 등을 조합
# 키워드: (반도체*장비) AND (인공지능+AI+머신러닝+딥러닝+신경망)

# 항목별 검색 파라미터 구성
# KIPRIS 항목별 검색 가이드에 따라 필드 설정
# astrtCont(초록), inventionTitle(발명의명칭) 등에 키워드 포함

QUERY_KEYWORDS = "반도체*(장비+제조+공정)*(인공지능+AI+학습+신경망+예측+최적화)"
# 실제 API 호출 시에는 쿼리 문법에 맞게 조정 필요.
# 여기서는 'freeSearch'(자유검색) 또는 'inventionTitle' + 'astrtCont' 조합 사용

def search_target_patents(page: int = 1, rows: int = 20) -> List[Dict]:
    """반도체 AI 관련 특허 검색"""
    params = {
        "astrtCont": "반도체*(인공지능+AI+학습+신경망)",  # 초록 검색
        "inventionTitle": "반도체", # 제목에 반도체 포함
        "ipcNumber": "H01L",        # 반도체 관련 IPC
        "numOfRows": rows,
        "pageNo": page,
        "patent": "true",           # 특허만 (실용신안 제외)
        "utility": "false",
        "lastvalue": ""             # 정렬 등 옵션
    }
    
    # 항목별 검색 호출
    data = kipris_get(PATH_ADVANCED_SEARCH, params)
    
    # 응답 파싱 (XML 구조에 따라 다름)
    try:
        items = data.get("response", {}).get("body", {}).get("items", {}).get("item", [])
        if isinstance(items, dict): # 결과가 1개일 경우 dict로 옴
            items = [items]
        return items
    except Exception as e:
        print(f"Parsing Error on page {page}: {e}")
        return []

In [5]:
# 3. 인용 문헌(Prior Art) 수집 함수

def get_prior_art(application_number: str) -> List[str]:
    """출원번호로 상세 정보를 조회하여 인용 문헌 번호 리스트 반환"""
    params = {
        "applicationNumber": application_number
    }
    data = kipris_get(PATH_BIBLIO_DETAIL, params)
    
    prior_arts = []
    try:
        # 응답 구조 파싱 (명세서 기준)
        # response > body > item > priorArtDocumentsInfoArray > priorArtDocumentsInfo
        item = data.get("response", {}).get("body", {}).get("item", {})
        p_array = item.get("priorArtDocumentsInfoArray", {}).get("priorArtDocumentsInfo", [])
        
        if isinstance(p_array, dict):
            p_array = [p_array]
            
        for p in p_array:
            # 심사관 인용 여부 확인 (Y인 것만 Ground Truth로 사용 권장)
            if p.get("examinerQuotationFlag") == "Y":
                doc_num = p.get("documentsNumber")
                if doc_num:
                    prior_arts.append(doc_num)
                    
    except Exception as e:
        # 인용 문헌이 없거나 구조가 다른 경우 무시
        pass
        
    return prior_arts

In [ ]:
# 4. 메인 수집 루프 (Target 검색 -> Prior Art 매핑) - 병렬 처리 적용
from concurrent.futures import ThreadPoolExecutor, as_completed

MAX_PAGES = 5  # 유효 데이터 확보를 위해 페이지 수 증가 (2 -> 5)
ROWS_PER_PAGE = 20
MAX_WORKERS = 5 # 병렬 작업 수

dataset = []

def process_patent(t):
    """단일 특허 처리 함수 (병렬 실행용)"""
    app_num = t.get("applicationNumber")
    if not app_num:
        return None
        
    # 상세 정보(인용 문헌) 조회
    prior_arts = get_prior_art(app_num)
    
    # [수정] 인용 문헌이 없는 경우(정답지가 없는 경우)는 데이터셋에서 제외
    if not prior_arts:
        return None
    
    return {
        "target_patent": {
            "application_number": app_num,
            "title": t.get("inventionTitle"),
            "abstract": t.get("astrtCont"),
            "ipc": t.get("ipcNumber"),
            "applicant": t.get("applicantName"),
            "date": t.get("applicationDate")
        },
        "ground_truth_prior_arts": prior_arts,
        "meta": {
            "source": "KIPRIS",
            "query_type": "semiconductor_ai"
        }
    }

print("=== 수집 시작 (병렬 처리) ===")

total_searched = 0

for page in tqdm(range(1, MAX_PAGES + 1), desc="Searching Pages"):
    targets = search_target_patents(page, ROWS_PER_PAGE)
    
    if not targets:
        print(f"No results on page {page}. Stopping.")
        break
    
    total_searched += len(targets)
    
    # 병렬로 상세 정보 수집
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        future_to_patent = {executor.submit(process_patent, t): t for t in targets}
        
        for future in tqdm(as_completed(future_to_patent), total=len(targets), desc=f"Processing Page {page}", leave=False):
            try:
                result = future.result()
                if result:
                    dataset.append(result)
            except Exception as e:
                print(f"Error processing patent: {e}")

print(f"=== 수집 완료 ===")
print(f"검색된 특허 수: {total_searched}건")
print(f"유효 데이터셋(인용문헌 보유) 수: {len(dataset)}건 (수율: {len(dataset)/total_searched*100:.1f}%)")

=== 수집 시작 (병렬 처리) ===


Searching Pages:   0%|          | 0/2 [00:00<?, ?it/s]

Processing Page 1:   0%|          | 0/20 [00:00<?, ?it/s]

Processing Page 2:   0%|          | 0/20 [00:00<?, ?it/s]

=== 수집 완료: 총 40건 ===


In [7]:
# 5. 데이터 저장 및 확인

# DataFrame 변환 (분석용)
df = pd.DataFrame([
    {
        "app_num": d["target_patent"]["application_number"],
        "title": d["target_patent"]["title"],
        "prior_art_count": len(d["ground_truth_prior_arts"]),
        "prior_arts": d["ground_truth_prior_arts"]
    }
    for d in dataset
])

print("데이터 미리보기:")
display(df.head())

# JSONL 저장 (실험용)
out_dir = REPO_ROOT / "data" / "processed"
out_dir.mkdir(parents=True, exist_ok=True)
out_path = out_dir / "kipris_semiconductor_ai_dataset.jsonl"

with open(out_path, "w", encoding="utf-8") as f:
    for entry in dataset:
        f.write(json.dumps(entry, ensure_ascii=False) + "\n")
        
print(f"데이터셋 저장 완료: {out_path}")

데이터 미리보기:


,app_num,title,prior_art_count,prior_arts
0,1020220021970,말단 디바이스의 인공지능 구축을 이용한 반도체 제조 시스템,0,[]
1,1020070020188,모터 구동 얼라이너를 구비하는 반도체 제조 설비 및 그의제어 방법,0,[]
2,1020240135833,학습형 반도체 공정 배기 제어 장치 및 방법,2,"[EP00875811 A3, JP2014194966 A]"
3,1020180084572,반도체 패키지,0,[]
4,1020200095371,공압 실린더의 동작 영상정보를 이용한 반도체 제조공정 감시 시스템과 감시 방법,0,[]


데이터셋 저장 완료: /home/arkwith/Dev/paper_data/data/processed/kipris_semiconductor_ai_dataset.jsonl
